In [1]:
import sys
from pathlib import Path

reporoot = Path(".").absolute().parent


In [2]:
sys.path.append(str(reporoot))


In [3]:
from c2st.check import c2st_


In [4]:
help(c2st_)

Help on function c2st_ in module c2st.check:

c2st_(X: numpy.ndarray, Y: numpy.ndarray, seed: int = 1, n_folds: int = 5, scoring: str = 'accuracy', z_score: bool = True, noise_scale: Optional[float] = None, verbosity: int = 0, clf_class=<class 'sklearn.ensemble._forest.RandomForestClassifier'>, clf_kwargs={}) -> numpy.ndarray
    Return accuracy of classifier trained to distinguish samples from supposedly
    two distributions <X> and <Y>. For details on the method, see [1,2].
    If the returned accuracy is 0.5, <X> and <Y> are considered to be from the
    same generating PDF, i.e. they can not be differentiated.
    If the returned accuracy is around 1., <X> and <Y> are considered to be from
    two different generating PDFs.
    
    Trains classifiers with N-fold cross-validation [3]. By default, a `RandomForestClassifier`
    by scikit-learn is used. This can be adopted using <clf_class> and
    <clf_kwargs> as in:
    
    ``` py
    clf = clf_class(random_state=seed, **clf_kwar

In [5]:
from __future__ import annotations
import numpy as np
from functools import partial
from numpy.random import default_rng
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import __version__ as sklversion
import time
FIXEDSEED = 1309

In [6]:
def nn_c2st_(
    X: np.ndarray,
    Y: np.ndarray,
    seed: int = FIXEDSEED,
    n_folds: int = 5,
    scoring: str = "accuracy",
    z_score: bool = True,
    noise_scale: Optional[float] = None,
    verbosity: int = 0,
) -> np.ndarray:

    ndim = X.shape[1]
    clf_class = MLPClassifier
    clf_kwargs = {
        "activation": "relu",
        "hidden_layer_sizes": (10 * ndim, 10 * ndim),
        "max_iter": 1000,
        "solver": "adam",
    }

    return c2st_(
        X,
        Y,
        seed,
        n_folds,
        scoring,
        z_score,
        noise_scale,
        verbosity,
        clf_class,
        clf_kwargs,
    )

def early_c2st_(
    X: np.ndarray,
    Y: np.ndarray,
    seed: int = FIXEDSEED,
    n_folds: int = 5,
    scoring: str = "accuracy",
    z_score: bool = True,
    noise_scale: Optional[float] = None,
    verbosity: int = 0,
) -> np.ndarray:

    ndim = X.shape[1]
    clf_class = MLPClassifier
    clf_kwargs = {
        "activation": "relu",
        "hidden_layer_sizes": (10 * ndim, 10 * ndim),
        "max_iter": 1000,
        "solver": "adam",
        "early_stopping": True,
        "n_iter_no_change": 50,
    }

    return c2st_(
        X,
        Y,
        seed,
        n_folds,
        scoring,
        z_score,
        noise_scale,
        verbosity,
        clf_class,
        clf_kwargs,
    )

In [7]:
NDIM = 10
max_nsamples = 8096
sample_sizes = [ 2**it for it in range(7,12)]
sample_sizes.append(max_nsamples)
RNG = default_rng(FIXEDSEED)
print(sample_sizes)

[128, 256, 512, 1024, 2048, 8096]


In [8]:
center_normal = partial(RNG.multivariate_normal, mean=np.zeros(NDIM), cov=np.eye(NDIM))
distributions = { 0. : center_normal}

for alpha in np.linspace(0,2,9):
    distributions[alpha] = partial(RNG.multivariate_normal, mean=np.zeros(NDIM) + alpha, cov=np.eye(NDIM))



In [9]:
center_samples = center_normal(size=max_nsamples)
samples = {}
for k,v in distributions.items():
    samples[k] = v(size=max_nsamples)

assert samples[0.].shape == (8096, 10)

In [10]:
rf_results = {}
rf_timings = {}
total = len(samples.values())*len(sample_sizes)
cnt = 0


for k,v in samples.items():
    for size in sample_sizes:

        try:
            start = time.time()
            scores = c2st_(X = center_samples[:size,...],
                          Y = v[:size,...],
                          n_folds=10)
            end = time.time()
        except Exception as ex:
            print(ex)
            continue

        if not k in rf_results.keys():
            rf_results[k] = []
            rf_timings[k] = []

        mean = np.mean(scores)
        std = np.std(scores)
        rf_results[k].append(list(scores))
        rf_timings[k].append(len(scores)*[end-start])
        cnt += 1

        print(f"{cnt}/{total}: {k}[{size},...] = {mean,std} ({end-start} seconds)")

1/54: 0.0[128,...] = (0.42153846153846153, 0.12246385772832794) (1.0709571838378906 seconds)


2/54: 0.0[256,...] = (0.47835595776772244, 0.07806781384146226) (1.415243148803711 seconds)


3/54: 0.0[512,...] = (0.5009232819341329, 0.048259547167832775) (2.2146804332733154 seconds)


4/54: 0.0[1024,...] = (0.48974414155906265, 0.019876207761461956) (4.505118370056152 seconds)


5/54: 0.0[2048,...] = (0.5007263402707377, 0.01581535273212949) (9.713266134262085 seconds)


...

50/54: 2.0[256,...] = (0.998076923076923, 0.0057692307692307826) (1.1295642852783203 seconds)


51/54: 2.0[512,...] = (0.9980487340567296, 0.00390258992495368) (1.59157395362854 seconds)


52/54: 2.0[1024,...] = (0.9985365853658535, 0.002235402778027241) (2.721519708633423 seconds)


53/54: 2.0[2048,...] = (0.998533603673445, 0.0019556928463501473) (5.600362300872803 seconds)


54/54: 2.0[8096,...] = (0.9986412508864639, 0.0006052481616375465) (29.05195903778076 seconds)


In [11]:
header = "ndims,mode,dist_sigma,nsamples,c2st_score,crossvalid,total_cvtime_sec,nfolds,sklearn_version"
with open("rf_results.csv","w") as ocsv:
    ocsv.write(header+"\n")
    for k in samples.keys():
        for idx in range(len(sample_sizes)):
            scores = rf_results[k][idx]
            timings = rf_timings[k][idx]
            for cvid in range(len(timings)):
                score = scores[cvid]
                timing = timings[cvid]
                row = f"{NDIM},rf,{k},{sample_sizes[idx]},{score},{cvid},{timing},10,{sklversion}\n"
                ocsv.write(row)

In [18]:
!head -n25 rf_results.csv|column -t -s","

ndims  mode  dist_sigma  nsamples  c2st_score           crossvalid  total_cvtime_sec    nfolds  sklearn_version
10     rf    0.0         128       0.6153846153846154   0           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.34615384615384615  1           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.34615384615384615  2           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.6153846153846154   3           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.4230769230769231   4           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.2692307692307692   5           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.48                 6           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.48                 7           1.0709571838378906  10      1.0.2
10     rf    0.0         128       0.24                 8           1.07

In [19]:
!tail -n25 rf_results.csv|column -t -s","

10  rf  2.0  1024  1.0                 5  2.721519708633423  10  1.0.2
10  rf  2.0  1024  1.0                 6  2.721519708633423  10  1.0.2
10  rf  2.0  1024  1.0                 7  2.721519708633423  10  1.0.2
10  rf  2.0  1024  1.0                 8  2.721519708633423  10  1.0.2
10  rf  2.0  1024  1.0                 9  2.721519708633423  10  1.0.2
10  rf  2.0  2048  1.0                 0  5.600362300872803  10  1.0.2
10  rf  2.0  2048  1.0                 1  5.600362300872803  10  1.0.2
10  rf  2.0  2048  0.9975609756097561  2  5.600362300872803  10  1.0.2
10  rf  2.0  2048  1.0                 3  5.600362300872803  10  1.0.2
10  rf  2.0  2048  1.0                 4  5.600362300872803  10  1.0.2
10  rf  2.0  2048  1.0                 5  5.600362300872803  10  1.0.2
10  rf  2.0  2048  0.9951100244498777  6  5.600362300872803  10  1.0.2
10  rf  2.0  2048  0.9951100244498777  7  5.600362300872803  10  1.0.2
10  rf  2.0  2048  1.0                 8  5.600362300872803  10  1.0.2
10  rf

In [23]:
early_results = {}
early_timings = {}
cnt = 0

for k,v in samples.items():
    for size in sample_sizes:

        try:
            start = time.time()
            scores = early_c2st_(center_samples[:size,...],
                                 v[:size,...],
                                 n_folds=10)
            end = time.time()
        except Exception as ex:
            print(ex)
            continue
        
        if not k in early_results.keys():
            early_results[k] = []
            early_timings[k] = []

        mean = np.mean(scores)
        std = np.std(scores)
        early_results[k].append(list(scores))
        early_timings[k].append(len(scores)*[end-start])
        # early_results[k].append((mean,std))
        # early_timings[k].append(end-start)
        cnt += 1

        print(f"{cnt}/{total}: {k}[{size},...] = {mean,std} ({end-start} seconds)")

1/54: 0.0[128,...] = (0.44984615384615384, 0.09478271632322777) (2.912928819656372 seconds)


2/54: 0.0[256,...] = (0.44528657616892914, 0.04583470991937402) (3.3204894065856934 seconds)


3/54: 0.0[512,...] = (0.46774224252807917, 0.045399203720113135) (6.249772787094116 seconds)


4/54: 0.0[1024,...] = (0.48537063605930175, 0.0309927203690838) (16.076107501983643 seconds)


5/54: 0.0[2048,...] = (0.5024312719899815, 0.02263994456097309) (24.020041942596436 seconds)


...

50/54: 2.0[256,...] = (0.9804298642533936, 0.021479613861054833) (6.342424631118774 seconds)


51/54: 2.0[512,...] = (0.9892727964972398, 0.012661869316598583) (6.657649755477905 seconds)


52/54: 2.0[1024,...] = (0.9965829746532758, 0.003124603599945947) (17.929768562316895 seconds)


53/54: 2.0[2048,...] = (0.9956079670821157, 0.0034144658947605116) (33.33112931251526 seconds)


54/54: 2.0[8096,...] = (0.9984562182112111, 0.0012114221736267189) (141.06727719306946 seconds)


In [35]:
with open("early_results.csv","w") as ocsv:
    ocsv.write(header+"\n")
    for k in samples.keys():
        for idx in range(len(sample_sizes)):
            scores = early_results[k][idx]
            timings = early_timings[k][idx]
            for cvid in range(len(timings)):
                score = scores[cvid]
                timing = timings[cvid]
                row = f"{NDIM},early,{k},{sample_sizes[idx]},{score},{cvid},{timing},10,{sklversion}\n"
                ocsv.write(row)
            

In [36]:
!head early_results.csv|column -t -s,

ndims  mode   dist_sigma  nsamples  c2st_score           crossvalid  total_cvtime_sec   nfolds  sklearn_version
10     early  0.0         128       0.4230769230769231   0           2.912928819656372  10      1.0.2
10     early  0.0         128       0.5384615384615384   1           2.912928819656372  10      1.0.2
10     early  0.0         128       0.46153846153846156  2           2.912928819656372  10      1.0.2
10     early  0.0         128       0.2692307692307692   3           2.912928819656372  10      1.0.2
10     early  0.0         128       0.5384615384615384   4           2.912928819656372  10      1.0.2
10     early  0.0         128       0.3076923076923077   5           2.912928819656372  10      1.0.2
10     early  0.0         128       0.48                 6           2.912928819656372  10      1.0.2
10     early  0.0         128       0.56                 7           2.912928819656372  10      1.0.2
10     early  0.0         128       0.4                  8           2.9

In [28]:
nn_results = {}
nn_timings = {}
cnt = 0

for k,v in samples.items():
    for size in sample_sizes:

        try:
            start = time.time()
            scores = nn_c2st_(center_samples[:size,...],
                              v[:size,...],
                              n_folds=10)
            end = time.time()
        except Exception as ex:
            print(ex)
            continue
        
        if not k in nn_results.keys():
            nn_results[k] = []
            nn_timings[k] = []

        mean = np.mean(scores)
        std = np.std(scores)
        nn_results[k].append(list(scores))
        nn_timings[k].append(len(scores)*[end-start])
        cnt += 1

        print(f"{cnt}/{total}: {k}[{size},...] = {mean,std} ({end-start} seconds)")

1/54: 0.0[128,...] = (0.43784615384615383, 0.11545474791695684) (11.881926774978638 seconds)


2/54: 0.0[256,...] = (0.4765837104072398, 0.05999918906760437) (23.398439407348633 seconds)


3/54: 0.0[512,...] = (0.46576242147344377, 0.03861967938800877) (46.214720726013184 seconds)


4/54: 0.0[1024,...] = (0.4751123864179818, 0.033294503971132865) (124.35642886161804 seconds)


5/54: 0.0[2048,...] = (0.48705766593118255, 0.01723649469724518) (285.923570394516 seconds)


...

50/54: 2.0[256,...] = (0.996078431372549, 0.007843137254901976) (6.071957111358643 seconds)


51/54: 2.0[512,...] = (0.9980487340567296, 0.0039025899249536806) (8.293025016784668 seconds)


52/54: 2.0[1024,...] = (0.9985365853658535, 0.003123475237772124) (20.737361669540405 seconds)


53/54: 2.0[2048,...] = (0.998046395133878, 0.002129578406386084) (37.38613438606262 seconds)


54/54: 2.0[8096,...] = (0.9980855046934932, 0.001085598195463115) (108.4162175655365 seconds)


In [33]:
with open("nn_results.csv","w") as ocsv:
    ocsv.write(header+"\n")
    for k in samples.keys():
        for idx in range(len(sample_sizes)):
            scores = nn_results[k][idx]
            timings = nn_timings[k][idx]
            for cvid in range(len(timings)):
                score = scores[cvid]
                timing = timings[cvid]
                row = f"{NDIM},nn,{k},{sample_sizes[idx]},{score},{cvid},{timing},10,{sklversion}\n"
                ocsv.write(row)

In [34]:
!head nn_results.csv|column -t -s,

ndims  mode  dist_sigma  nsamples  c2st_score           crossvalid  total_cvtime_sec    nfolds  sklearn_version
10     nn    0.0         128       0.34615384615384615  0           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.6538461538461539   1           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.5384615384615384   2           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.23076923076923078  3           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.38461538461538464  4           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.38461538461538464  5           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.56                 6           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.48                 7           11.881926774978638  10      1.0.2
10     nn    0.0         128       0.4                  8           11.8